In [ ]:
# default_exp mediumapi

# MediumAPI

> This module is as close as possible to the barebone [Medium API](https://github.com/Medium/medium-api-docs#33-posts) but written in python instead of GET/POST commands. Not all the kinds requests are implemented, only the ones that are useful to post an article or an image

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
from requests import get, post, HTTPError
import os

In [ ]:
#export
def base_request():
    try:
        response = get(url)
        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        print('Successfull GET Request')

## Fetch User Data

The basic user data can be requested by passing the Medium Integration token via a GET request. The Medium Integration Token needs to be requested from the writer's Medium profile page. For now, this token should be stored as an environment variable under the user's `$HOME/.profile` file:
```md
export MEDIUM_TOKEN=<the-token>
```
I may consider using keyring to store the token locally as that may more user friendly.

In [ ]:
#export
def auth_header(token = None, test = False):
    if token is None: 
        token = os.getenv('MEDIUM_TOKEN') if not test else os.getenv('MEDIUM_TOKEN_TEST')
    return {'Authorization': f"Bearer {token}"}

In [ ]:
#export
def fetch_user_data(test = False):
    return get("https://api.medium.com/v1/me", headers = auth_header(test = test))

In [ ]:
a = fetch_user_data()

In [ ]:
a.json()

{'data': {'id': '1e344db7dfd2a8efa9698a758030e05e28ff4c396f1f87f003704e0a8a80b9656',
  'username': 'lucha6',
  'name': 'Luis Chaves',
  'url': 'https://medium.com/@lucha6',
  'imageUrl': 'https://cdn-images-1.medium.com/fit/c/400/400/0*rM2cEh8f6ZQMOAZK.jpg'}}

Which is equivalent to the `curl` alternative:

In [ ]:
!curl -s -H "Authorization: Bearer $MEDIUM_TOKEN" https://api.medium.com/v1/me | jq

{
  "data": {
    "id": "1e344db7dfd2a8efa9698a758030e05e28ff4c396f1f87f003704e0a8a80b9656",
    "username": "lucha6",
    "name": "Luis Chaves",
    "url": "https://medium.com/@lucha6",
    "imageUrl": "https://cdn-images-1.medium.com/fit/c/400/400/0*rM2cEh8f6ZQMOAZK.jpg"
  }
}


In [ ]:
assert 'data' in fetch_user_data().json().keys()

## Get User ID


In [ ]:
#export
def get_user_id(test = False):
    return fetch_user_data(test).json()['data']['id']

In [ ]:
get_user_id()

'1e344db7dfd2a8efa9698a758030e05e28ff4c396f1f87f003704e0a8a80b9656'

In [ ]:
get_user_id(test = True)

'16eaaf7a06cf688353bd560c48dd5ef7fa1f0350adc3d0d211f596e698115e308'

## Fetch User Publications

We can also request the user's publications. In Medium's definition a publication is not an article but rather an editorial-like group under which articles are written (e.g. Towards Data Science, Elemental AI...)

In [ ]:
#export
def fetch_publications():
    return get(f"https://api.medium.com/v1/users/{get_user_id()}/publications", headers = auth_header())

In [ ]:
fetch_publications().json()

{'data': [{'id': '7f60cf5620c9',
   'name': 'Towards Data Science',
   'description': 'Your home for data science. A Medium publication sharing concepts, ideas and codes.',
   'url': 'https://medium.com/towards-data-science',
   'imageUrl': 'https://cdn-images-1.medium.com/fit/c/400/400/1*hVxgUA6kP-PgL5TJjuyePg.png'},
  {'id': '4b3a1ed4f11c',
   'name': 'JavaScript in Plain English',
   'description': 'New JavaScript and Web Development articles every day.',
   'url': 'https://medium.com/javascript-in-plain-english',
   'imageUrl': 'https://cdn-images-1.medium.com/fit/c/400/400/1*4F1ZAI1i-eruO4PnbNvTZg@2x.png'},
  {'id': '261e46dce6ca',
   'name': '<pretty/code>',
   'description': 'Topics centered around Ruby, Rails, Coffeescript, Vim, Tmux and Productivity.',
   'url': 'https://medium.com/raise-coffee',
   'imageUrl': 'https://cdn-images-1.medium.com/fit/c/400/400/1*pLo2lxSseBKg09Nc_1EOlw.png'},
  {'id': '3a8144eabfe3',
   'name': 'HackerNoon.com',
   'description': 'Elijah McClain, 

In [ ]:
assert 'data' in fetch_publications().json().keys()

## Post an Article

As easily an article can be submitted. Many options are available which can be explored in the [Medium API official docs](https://github.com/Medium/medium-api-docs#33-posts). The possible parameters and parameter values are as follow:


| Parameter       | Type         | Required?  | Description                                     |
| -------------   |--------------|------------|-------------------------------------------------|
| title           | string       | required   | The title of the post. Note that this title is used for SEO and when rendering the post as a listing, but will not appear in the actual post—for that, the title must be specified in the `content` field as well. Titles longer than 100 characters will be ignored. In that case, a title will be synthesized from the first content in the post when it is published.  |
| contentFormat   | string       | required   | The format of the "content" field. There are two valid values, "html", and "markdown" |
| content         | string       | required   | The body of the post, in a valid, semantic, HTML fragment, or Markdown. Further markups may be supported in the future. For a full list of accepted HTML tags, see [here](https://medium.com/@katie/a4367010924e). If you want your title to appear on the post page, you must also include it as part of the post content.                |
| tags            | string array | optional   | Tags to classify the post. Only the first three will be used. Tags longer than 25 characters will be ignored.                                        |
| canonicalUrl    | string       | optional   | The original home of this content, if it was originally published elsewhere.                         |
| publishStatus   | enum         | optional   | The status of the post. Valid values are “public”, “draft”, or “unlisted”. The default is “public”.  |
| license         | enum         | optional   | The license of the post. Valid values are “all-rights-reserved”, “cc-40-by”, “cc-40-by-sa”, “cc-40-by-nd”, “cc-40-by-nc”, “cc-40-by-nc-nd”, “cc-40-by-nc-sa”, “cc-40-zero”, “public-domain”. The default is “all-rights-reserved”. |
| notifyFollowers | bool         | optional   | Whether to notifyFollowers that the user has published. |




The default `publishStatus` for posting articles will __always__ be set to 'draft' because that is how I would always like to use this API. 

In [ ]:
#export
def post_article(
    title,
    content,
    contentFormat = 'markdown',
    tags = None,
    canonicalUrl = None,
    publishStatus = 'draft',
    license = None,
    notifyFollowers = False,
    test = False # internal
):
    data = {
        'title': title,
        'contentFormat': contentFormat,
        'content': content,
        'tags': tags,
        'canonicalUrl': canonicalUrl,
        'publishStatus': publishStatus,
        'license': license,
        'notifyFollowers': notifyFollowers
    }
    
    return post(f"https://api.medium.com/v1/users/{get_user_id(test)}/posts",
                data = data,
                headers = auth_header(test = test))

Which is equivalent to:

In [ ]:
!curl \
  -X POST \
  -H "Accept: application/vnd.github.v3+json" \
  https://api.github.com/gists \
  -d '{"files":{"CONTRIBUTING.md"}}'

{
  "message": "Requires authentication",
  "documentation_url": "https://docs.github.com/rest/reference/gists#create-a-gist"
}


This command posts an article via a simple POST request. If the article is correctly submitted a JSON response like the below will be returned

### Posting a string of text

In [ ]:
my_post = post_article('Test from nb',
             '# Markdown title \n\n markdown text',
             test = True)

In [ ]:
my_post.json()

{'data': {'id': 'f0d5723f4bd4',
  'title': 'Test from nb',
  'authorId': '16eaaf7a06cf688353bd560c48dd5ef7fa1f0350adc3d0d211f596e698115e308',
  'url': 'https://medium.com/@luischaveshuntington/f0d5723f4bd4',
  'canonicalUrl': '',
  'publishStatus': 'draft',
  'license': '',
  'licenseUrl': 'https://policy.medium.com/medium-terms-of-service-9db0094a1e0f',
  'tags': []}}

The request should return the [HTTP 201 code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/201)

### Posting a text document

In [ ]:
my_post_from_file = post_article('Test from file',open('../tests/LEARNING.md', 'rb').read(),test = True)

In [ ]:
my_post_from_file.json()

{'data': {'id': 'd0daa0d04215',
  'title': 'Test from file',
  'authorId': '16eaaf7a06cf688353bd560c48dd5ef7fa1f0350adc3d0d211f596e698115e308',
  'url': 'https://medium.com/@luischaveshuntington/d0daa0d04215',
  'canonicalUrl': '',
  'publishStatus': 'draft',
  'license': '',
  'licenseUrl': 'https://policy.medium.com/medium-terms-of-service-9db0094a1e0f',
  'tags': []}}

In [ ]:
assert my_post_from_file.ok
assert my_post_from_file.status_code == 201 

## Post/Upload an image

In [ ]:
#export
import random
from io import BytesIO

def post_image(filename = None, img = None, test = False):
    """
    filename: needs to be a valid image file path supported my Medium
    img: can be a binary image representation
    """
    if img is None:
        img = open(filename, 'rb')
    else: 
        filename = str(random.randint(1e4, 1e5))
    files = {
        'image': (os.path.basename(filename), img, 'image/png')
    }
    img.close() if img is None else 1
    return post(f"https://api.medium.com/v1/images",
                headers = auth_header(test = test),
                files = files)

## Upload image from file

In [ ]:
my_img = post_image('../tests/github-logo.png', test = True)
my_img.json()

{'data': {'url': 'https://cdn-images-1.medium.com/proxy/1*sV7tva-728oySeOUL0-vOw.png',
  'md5': 'sV7tva-728oySeOUL0-vOw'}}

In [ ]:
assert my_img.ok
assert my_img.status_code == 201

### Upload image as byte stream

In [ ]:
from io import BytesIO
img = BytesIO(open('../tests/github-logo.png', 'rb').read()).getvalue()
print(f"Which is some sort of byte stream: like {img[:10]}")

Which is some sort of byte stream: like b'\x89PNG\n\n\x1a\n\x00\x00'


In [ ]:
image_from_bytes = post_image(filename = 'github-logo.png', img = img, test = True).json()
image_from_bytes

{'data': {'url': 'https://cdn-images-1.medium.com/proxy/1*sV7tva-728oySeOUL0-vOw.png',
  'md5': 'sV7tva-728oySeOUL0-vOw'}}

It may not become apparent why this functionality is useful right now. Being able to upload a file as a binary stream is actually really useful because we can avoid saving the images to memory and upload them directly to Medium, the `nbconvert` modules that we will be using later represent images in such intermediate states

Which as a `curl` call would be:

In [ ]:
!curl -X POST https://api.medium.com/v1/images \
	-H "Authorization: Bearer $MEDIUM_TOKEN_TEST" \
	-F 'name="image"; filename="github-logo.png" ; type="image/png";' \
	-F 'image=@LEARNING/github-logo.png' | jq

curl: (26) Failed to open/read local data from file/application


## Posting an article with an image

### Post with online image links

It turns out that uploading a file with an online image may be easier

In [ ]:
with open('../tests/test-with-online-image.md', 'rb') as article:
    my_post_from_file_with_online_image = post_article(
        'Test from file with online image',
        article.read(),
        test = True)

In [ ]:
my_post_from_file_with_online_image.json()

{'data': {'id': '2c7dafd0689b',
  'title': 'Test from file with online image',
  'authorId': '16eaaf7a06cf688353bd560c48dd5ef7fa1f0350adc3d0d211f596e698115e308',
  'url': 'https://medium.com/@luischaveshuntington/2c7dafd0689b',
  'canonicalUrl': '',
  'publishStatus': 'draft',
  'license': '',
  'licenseUrl': 'https://policy.medium.com/medium-terms-of-service-9db0094a1e0f',
  'tags': []}}

For those reading this code above, there is no way for me to show that the draft was posted succesfully and that the image is displayed correctly as the draft is posted to my account, but trust me it is posted correctly. Where as obtaining a correct JSON back is a good sign that the article was posted succesfully, the posted draft still needs to be verified and potentially modified.

In [ ]:
with open('../tests/test-with-offline-image.md', 'rb') as article:
    my_post_from_file_with_offline_image = post_article(
        'Test from file with offline image',
        article.read(),
        test = True)

In [ ]:
my_post_from_file_with_offline_image.json()

{'data': {'id': '47547e87d4f2',
  'title': 'Test from file with offline image',
  'authorId': '16eaaf7a06cf688353bd560c48dd5ef7fa1f0350adc3d0d211f596e698115e308',
  'url': 'https://medium.com/@luischaveshuntington/47547e87d4f2',
  'canonicalUrl': '',
  'publishStatus': 'draft',
  'license': '',
  'licenseUrl': 'https://policy.medium.com/medium-terms-of-service-9db0094a1e0f',
  'tags': []}}

In this case, we referred to a local image in our markdown document as opposed to one found oneline. When we post an article with references to offline/local images, the medium Markdown renderer won't recognise the path to those images and will faily to display the image. __What would need to be done__ in this case is to get the local image paths, upload them with the `post_image()` function and then replace the local path reference by the one in the Medium DB.

In [ ]:
#hide
from nbdev.export import *
notebook2script()